In [1]:
from filterpy.kalman import KalmanFilter
from sort import *


In [12]:
!yolo task=detect mode=predict model="best (3).pt" conf=0.75 source="Field Dataset/train/images/960x0 (10).jpg"

Ultralytics YOLOv8.0.190  Python-3.8.18 torch-2.0.1+cpu CPU (AMD Ryzen 5 3550H with Radeon Vega Mobile Gfx)
Model summary (fused): 268 layers, 43607379 parameters, 0 gradients, 164.8 GFLOPs

image 1/1 d:\AI Enigma Contest\Light Detection\Field Dataset\train\images\960x0 (10).jpg: 256x480 1 light, 749.4ms
Speed: 0.0ms preprocess, 749.4ms inference, 0.0ms postprocess per image at shape (1, 3, 256, 480)
Results saved to runs\detect\predict3
 Learn more at https://docs.ultralytics.com/modes/predict


In [13]:
from ultralytics import YOLO
import cv2
import cvzone
import math

# cap = cv2.VideoCapture(1)  # For Webcam
# cap.set(3, 1280)
# cap.set(4, 720)
cap = cv2.VideoCapture("960x0_(5)_V1.mp4")  # For Video


model = YOLO("best (3).pt")

classNames = ["light"]


while True:
    new_frame_time = time.time()
    success, img = cap.read()
    results = model(img, stream=True)
    for r in results:
        boxes = r.boxes
        for box in boxes:
            # Bounding Box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            # cv2.rectangle(img,(x1,y1),(x2,y2),(255,0,255),3)
            w, h = x2 - x1, y2 - y1
            cvzone.cornerRect(img, (x1, y1, w, h))
            # Confidence
            conf = math.ceil((box.conf[0] * 100)) / 100
            # Class Name
            cls = int(box.cls[0])

            cvzone.putTextRect(img, f'{classNames[cls]} {conf}', (max(0, x1), max(35, y1)), scale=1, thickness=1)


    cv2.imshow("Image", img)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break
cap.release()
  
# Closes all the frames
cv2.destroyAllWindows()



0: 256x480 2 lights, 744.9ms
Speed: 0.0ms preprocess, 744.9ms inference, 0.0ms postprocess per image at shape (1, 3, 256, 480)

0: 256x480 2 lights, 715.3ms
Speed: 0.0ms preprocess, 715.3ms inference, 0.0ms postprocess per image at shape (1, 3, 256, 480)

0: 256x480 2 lights, 719.6ms
Speed: 1.0ms preprocess, 719.6ms inference, 0.0ms postprocess per image at shape (1, 3, 256, 480)

0: 256x480 2 lights, 717.6ms
Speed: 0.9ms preprocess, 717.6ms inference, 0.0ms postprocess per image at shape (1, 3, 256, 480)

0: 256x480 2 lights, 696.0ms
Speed: 1.0ms preprocess, 696.0ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 480)

0: 256x480 2 lights, 749.8ms
Speed: 0.9ms preprocess, 749.8ms inference, 0.0ms postprocess per image at shape (1, 3, 256, 480)

0: 256x480 2 lights, 723.6ms
Speed: 2.0ms preprocess, 723.6ms inference, 0.0ms postprocess per image at shape (1, 3, 256, 480)

0: 256x480 2 lights, 711.5ms
Speed: 1.1ms preprocess, 711.5ms inference, 0.0ms postprocess per image at

In [4]:
from ultralytics import YOLO
import cv2
import cvzone
import math
from filterpy.kalman import KalmanFilter
import argparse
import time
import glob
from skimage import io
import matplotlib.patches as patches
import matplotlib.pyplot as plt

import os
import numpy as np
import matplotlib
from sort import *

# cap = cv2.VideoCapture(1)  # For Webcam
# cap.set(3, 1280)
# cap.set(4, 720)
cap = cv2.VideoCapture("960x0_(5)_V1.mp4")  # For Video


model = YOLO("best (3).pt")

classNames = ['light']
tracker = Sort(max_age=20, min_hits=3, iou_threshold=0.3)
zone1 = 0
zone2 = 0
zone3 = 0
zone4 = 0

limits = [500, 300, 1000, 300]
limits2 = [0, 300, 500, 300]
limits3 = [500, 150, 1000, 150]
limits4 = [0, 150, 500, 150]

while True:
    success, img = cap.read()
    results = model(img, stream=True)
    detections = np.empty((0, 5))
    for r in results:
        boxes = r.boxes
        for box in boxes:
            # Bounding Box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            # cv2.rectangle(img,(x1,y1),(x2,y2),(255,0,255),3)
            w, h = x2 - x1, y2 - y1
            cvzone.cornerRect(img, (x1, y1, w, h))
            # Confidence
            conf = math.ceil((box.conf[0] * 100)) / 100
            # Class Name
            cls = int(box.cls[0])
            if conf > 0.5:
                currentArray = np.array([x1, y1, x2, y2, conf])
                detections = np.vstack((detections, currentArray))
    resultsTracker = tracker.update(detections)

#     cv2.line(img, (limits[0], limits[1]), (limits[2], limits[3]), (0, 0, 255), 5)
#     cv2.line(img, (limits2[0], limits2[1]), (limits2[2], limits2[3]), (255, 0, 0), 5)
#     cv2.line(img, (limits3[0], limits3[1]), (limits3[2], limits3[3]), (0, 255, 0), 5)
#     cv2.line(img, (limits4[0], limits4[1]), (limits4[2], limits4[3]), (255, 0, 255), 5)

    
    for result in resultsTracker:
        x1, y1, x2, y2, id = result
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        print(result)
        w, h = x2 - x1, y2 - y1
        cvzone.cornerRect(img, (x1, y1, w, h), l=9, rt=2, colorR=(255, 0, 255))
        cvzone.putTextRect(img, f' {int(id)}', (max(0, x1), max(35, y1)),
                           scale=2, thickness=3, offset=10)

        cx, cy = x1 + w // 2, y1 + h // 2
        cv2.circle(img, (cx, cy), 5, (255, 0, 255), cv2.FILLED)
        if limits[0] < cx < limits[2] and limits[1] - 150 < cy < limits[1] + 150:
            zone1 = zone1 + 1
#             cv2.line(img, (limits[0], limits[1]), (limits[2], limits[3]), (0, 255, 0), 5)
        if limits2[0] < cx < limits2[2] and limits2[1] - 150 < cy < limits2[1] + 150:
            zone2 = zone2 + 1
#             cv2.line(img, (limits2[0], limits2[1]), (limits2[2], limits2[3]), (0, 255, 0), 5)
        if limits3[0] < cx < limits3[2] and limits3[1] - 150 < cy < limits3[1] + 150:
            zone3 = zone3 + 1
#             cv2.line(img, (limits3[0], limits3[1]), (limits3[2], limits3[3]), (0, 255, 0), 5)
        if limits4[0] < cx < limits4[2] and limits4[1] - 150 < cy < limits4[1] + 150:
            zone4 = zone4 + 1
            continue
#             cv2.line(img, (limits4[0], limits4[1]), (limits4[2], limits4[3]), (0, 255, 0), 5)
    
    cv2.putText(img,str(f"Zone 1: {zone4}s"),(155,100),cv2.FONT_HERSHEY_PLAIN,3,(50,50,255),3)
    cv2.putText(img,str(f"Zone 2: {zone3}s"),(555,100),cv2.FONT_HERSHEY_PLAIN,3,(50,50,255),3)
    cv2.putText(img,str(f"Zone 3: {zone2}s"),(155,400),cv2.FONT_HERSHEY_PLAIN,3,(50,50,255),3)
    cv2.putText(img,str(f"Zone 4: {zone1}s"),(555,400),cv2.FONT_HERSHEY_PLAIN,3,(50,50,255),3)

    cv2.imshow("Sunlight amount detector", img)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break
cap.release()
  
# Closes all the frames
cv2.destroyAllWindows()



0: 256x480 2 lights, 760.0ms
Speed: 15.4ms preprocess, 760.0ms inference, 0.0ms postprocess per image at shape (1, 3, 256, 480)



[          0           2         503         195           2]
[        477           4         958         208           1]


0: 256x480 2 lights, 729.4ms
Speed: 4.0ms preprocess, 729.4ms inference, 0.0ms postprocess per image at shape (1, 3, 256, 480)



[          0           2         503         195           2]
[        477           4         958         208           1]


0: 256x480 2 lights, 715.6ms
Speed: 2.0ms preprocess, 715.6ms inference, 0.0ms postprocess per image at shape (1, 3, 256, 480)



...

0: 256x480 2 lights, 938.0ms
Speed: 2.0ms preprocess, 938.0ms inference, 0.0ms postprocess per image at shape (1, 3, 256, 480)
WARNING  'source' is missing. Using 'source=C:\Users\mdsaz\.conda\envs\likhon\Lib\site-packages\ultralytics\assets'.



[     5.1762      2.7656      958.51      213.08           3]


image 1/2 C:\Users\mdsaz\.conda\envs\likhon\Lib\site-packages\ultralytics\assets\bus.jpg: 480x384 8 lights, 1343.8ms
image 2/2 C:\Users\mdsaz\.conda\envs\likhon\Lib\site-packages\ultralytics\assets\zidane.jpg: 288x480 3 lights, 926.6ms
Speed: 0.6ms preprocess, 1135.2ms inference, 0.0ms postprocess per image at shape (1, 3, 288, 480)


[     27.948      25.265      903.87      236.36           3]


error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:971: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'
